In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tqdm as tqdm
import shutil

import time
from contextlib import contextmanager
import os, argparse
import cv2, numpy as np
# from sklearn.externals import joblib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import re
import gc

import csv
import glob
# set the matplotlib backend so figures can be saved in the background
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
# import the necessary packages

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.metrics import classification_report

from imutils import paths
import math
import numpy as np
import pickle
import operator
from operator import itemgetter
from itertools import zip_longest
from collections import defaultdict
import json
import joblib
from tqdm import tqdm
import pandas as pd
from nltk.tokenize.treebank import TreebankWordTokenizer
import pandas as pd
import seaborn as sns
import datetime

%matplotlib inline

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')


In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU, add, Conv2D, Reshape
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, multiply
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras import utils
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image, text, sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

In [3]:
from vqa_util.vqa_model import build_model

In [4]:
tok = text.Tokenizers(filters='')
# load from disk
with open('./something.pkl', 'rb') as f:
    tok = joblib.load(f)

<function vqa_util.vqa_model.build_model(max_answers, max_seq_len, vocab_size, dim_d, dim_k, l_rate, d_rate, reg_value)>

In [ ]:
# load from disk
with open('./something.pkl', mode='rb') as f:
    question_data_train, question_data_val = pickle.load(f)

In [ ]:
# load from disk
with open('./something.pkl', 'rb') as f:
    labelencoder = joblib.load(f)

In [ ]:
def get_answers_matrix(answers, encoder):
	'''
	One-hot-encodes the answers

	Input:
		answers:	list of answer
		encoder:	a scikit-learn LabelEncoder object
  
	Output:
		A numpy array of shape (# of answers, # of class)
	'''
	y = encoder.transform(answers) #string to numerical class
	nb_classes = encoder.classes_.shape[0]
	Y = utils.to_categorical(y, nb_classes)
	return Y

### Prepare data matrices

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size= 0.25,random_state=42)

for train_index, val_index in sss.split(images_train, answer_train):
    TRAIN_INDEX = train_index
    VAL_INDEX = val_index

In [ ]:
# image data
image_list_tr, image_list_vl = np.array(images_train)[TRAIN_INDEX.astype(int)], np.array(images_train)[VAL_INDEX.astype(int)]

In [ ]:
# question data
question_tr, question_vl = question_data_train[TRAIN_INDEX], question_data_train[VAL_INDEX]

In [ ]:
# answer data
answer_matrix = get_answers_matrix(answer_train, labelencoder)
answer_tr, answer_vl = answer_matrix[TRAIN_INDEX], answer_matrix[VAL_INDEX]

## Model Creation

In [ ]:
BATCH_SIZE = 300
BUFFER_SIZE = 5000

In [ ]:
# params 1
max_answers = 1000
max_seq_len = 22
vocab_size  = len(tok.word_index) + 1
EPOCHS      = 60

dim_d       = 512
dim_k       = 256
l_rate      = 1e-4
d_rate      = 0.5
reg_value   = 0.01

base_path = './temp'

In [ ]:
# create model
model = build_model(max_answers, max_seq_len, vocab_size, dim_d, dim_k, l_rate, d_rate, reg_value)
model.summary()

Select loss and optimizer

In [ ]:
steps_per_epoch = int(np.ceil(len(image_list_tr)/BATCH_SIZE))
boundaries      = [50*steps_per_epoch]
values          = [l_rate, l_rate/10]

In [ ]:
# we reduce the l_rate after 50th epoch (from 1e-4 to 1e-5)
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
optimizer        = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)

loss_object      = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='auto')

In [ ]:
checkpoint_directory = base_path+"/training_checkpoints/"+str(l_rate)+"_"+str(dim_k)
SAVE_CKPT_FREQ = 5
ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, checkpoint_directory, max_to_keep=3)

Create stateful metrics that can be used to accumulate values during training and logged at any point:

In [ ]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
val_loss = tf.keras.metrics.Mean('val_loss', dtype=tf.float32)
train_score = F1Score(num_classes=max_answers, average='micro', name='train_score')
val_score = F1Score(num_classes=max_answers, average='micro', name='val_score')

Configure the tensorboard

In [ ]:
train_log_dir = base_path+'/logs/'+str(l_rate)+"_"+str(dim_k)+'/train'
val_log_dir   = base_path+'/logs/'+str(l_rate)+"_"+str(dim_k)+'/validation'

train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

Define the training and test functions

In [6]:
# @tf.function
def train_step(model, img, ques, ans, optimizer):
    with tf.GradientTape() as tape:
        # forward pass
        predictions = model([img, ques], training=True)
        loss = loss_object(ans, predictions)

    # backward pass
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # record results
    train_loss(loss)
    train_score(ans, predictions)

    # all gradients
    grads_ = list(zip(grads, model.trainable_variables))
    return grads_

def test_step(model, img, ques, ans):
    predictions = model([img, ques])
    loss = loss_object(ans, predictions)

    # record results
    val_loss(loss)
    val_score(ans, predictions)

In [ ]:
if manager.latest_checkpoint:
    ckpt.restore(manager.latest_checkpoint)
    print("Restored from {}".format(manager.latest_checkpoint))
    START_EPOCH = int(manager.latest_checkpoint.split('-')[-1]) * SAVE_CKPT_FREQ
    print("Resume training from epoch: {}".format(START_EPOCH))
else:
    print("Initializing from scratch")
    START_EPOCH = 0

In [7]:
for epoch in range(START_EPOCH, EPOCHS):

    start = time.time()

    for img, ques, ans in (dataset_tr):
        grads = train_step(model, img, ques, ans, optimizer)

    # tensorboard  
    with train_summary_writer.as_default():
        # Create a summary to monitor cost tensor
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        # Create a summary to monitor accuracy tensor
        tf.summary.scalar('f1_score', train_score.result(), step=epoch)
        # Create summaries to visualize weights
        for var in model.trainable_variables:
            tf.summary.histogram(var.name, var, step=epoch)
        # Summarize all gradients
        for grad, var in grads:
            tf.summary.histogram(var.name + '/gradient', grad, step=epoch)

    for img, ques, ans in (dataset_vl):
        test_step(model, img, ques, ans)

    # tensorboard
    with val_summary_writer.as_default():
        # Create a summary to monitor cost tensor
        tf.summary.scalar('loss', val_loss.result(), step=epoch)
        # Create a summary to monitor accuracy tensor
        tf.summary.scalar('f1_score', val_score.result(), step=epoch)

    template = 'Epoch {}, loss: {:.4f}, f1_score: {:.4f}, val loss: {:.4f}, val f1_score: {:.4f}, time: {:.0f} sec'
    print (template.format(epoch + 1,
                         train_loss.result(), 
                         train_score.result(),
                         val_loss.result(), 
                         val_score.result(),
                         (time.time() - start)))

    # Reset metrics every epoch
    train_loss.reset_states()
    train_score.reset_states()
    val_loss.reset_states()
    val_score.reset_states()

    # save checkpoint every SAVE_CKPT_FREQ step
    ckpt.step.assign_add(1)
    if int(ckpt.step) % SAVE_CKPT_FREQ == 0:
        manager.save()
        print('Saved checkpoint.')

NameError: name 'START_EPOCH' is not defined

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard